In [23]:
import sqlite3
import numpy as np
import io
import sqlalchemy
from sqlalchemy import orm
from pprint import pprint

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

In [24]:
sqlalchemy.orm

<module 'sqlalchemy.orm' from 'C:\\Users\\mille\\Miniconda3\\envs\\islands_py36\\lib\\site-packages\\sqlalchemy\\orm\\__init__.py'>

In [2]:
from uuid import uuid4
u = uuid4()

In [3]:
u.int

49660804755689211777704840428198007265

In [7]:
sqlalchemy.__version__

'1.2.11'

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:', echo=True)
Session = sessionmaker(bind=engine)
engine, Session

(Engine(sqlite:///:memory:),
 sessionmaker(class_='Session', bind=Engine(sqlite:///:memory:), autoflush=True, autocommit=False, expire_on_commit=True))

In [9]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Sequence, ForeignKey

Base = declarative_base()

class Entity(Base):
    __tablename__ = 'entities'

    eid = Column(Integer, autoincrement='auto', primary_key=True)
    name = Column(String(30))

    def __repr__(self):
        return "<Entity(id='{}', name='{}')>".format(self.id, self.name)

class Description(Base):
    __tablename__ = 'description'

    entity = Column(Integer, ForeignKey('entities.eid'), primary_key=True)
    short_desc = Column(String(40))
    long_desc = Column(String)


In [10]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2018-10-27 12:06:10,004 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2018-10-27 12:06:10,004 INFO sqlalchemy.engine.base.Engine ()


2018-10-27 12:06:10,020 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


2018-10-27 12:06:10,020 INFO sqlalchemy.engine.base.Engine ()


2018-10-27 12:06:10,020 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("entities")


...

2018-10-27 12:06:10,060 INFO sqlalchemy.engine.base.Engine ()


2018-10-27 12:06:10,063 INFO sqlalchemy.engine.base.Engine COMMIT


2018-10-27 12:06:10,064 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE description (
	entity INTEGER NOT NULL, 
	short_desc VARCHAR(40), 
	long_desc VARCHAR, 
	PRIMARY KEY (entity), 
	FOREIGN KEY(entity) REFERENCES entities (eid)
)




2018-10-27 12:06:10,067 INFO sqlalchemy.engine.base.Engine ()


2018-10-27 12:06:10,069 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
session = Session()
db = session
player = Entity(name='John the Player')
orc = Entity(name='Orc01')
session.add_all([player, orc])
session.flush()

2018-10-27 12:07:18,441 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2018-10-27 12:07:18,457 INFO sqlalchemy.engine.base.Engine INSERT INTO entities (name) VALUES (?)


2018-10-27 12:07:18,457 INFO sqlalchemy.engine.base.Engine ('John the Player',)


2018-10-27 12:07:18,473 INFO sqlalchemy.engine.base.Engine INSERT INTO entities (name) VALUES (?)


2018-10-27 12:07:18,473 INFO sqlalchemy.engine.base.Engine ('Orc01',)


In [17]:
player.eid

3

In [18]:
d0 = Description(entity=player.eid, short_desc='Lanky and balding.')
d1 = Description(entity=orc.eid, short_desc='Green and stout.')
session.add_all([d0, d1])
session.flush()

2018-10-27 12:07:21,437 INFO sqlalchemy.engine.base.Engine INSERT INTO description (entity, short_desc, long_desc) VALUES (?, ?, ?)


2018-10-27 12:07:21,440 INFO sqlalchemy.engine.base.Engine ((3, 'Lanky and balding.', None), (4, 'Green and stout.', None))


Numpy objects in sqlite3, thanks to [stackoverflow](http://stackoverflow.com/questions/18621513/python-insert-numpy-array-into-sqlite3-database)

In [25]:
class Position(Entity):
    entity = orm.PrimaryKey(int)
    _pos = orm.Optional(bytes)

    @property
    def pos(self):
        if not hasattr(self, "_cached") or self._needs_update:
            out = io.BytesIO(self._pos)
            out.seek(0)
            self._cached = np.load(out)
            self._needs_update = False

        return self._cached

    @pos.setter
    def pos(self, new_value):
        """
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        self._needs_update = True
        out = io.BytesIO()
        np.save(out, new_value)
        out.seek(0)
        self._pos = out.read()



AttributeError: module 'sqlalchemy.orm' has no attribute 'PrimaryKey'

In [15]:
db.generate_mapping(create_tables=True)

NameError: name 'db' is not defined

In [105]:
e1 = GameEntity(name='Entity1') # id not generated until stored in db
db.commit()
pos = Position(entity=e1.id)


In [122]:
pos.pos = np.zeros(3)

In [124]:
pos._pos

b"\x93NUMPY\x01\x00F\x00{'descr': '<f8', 'fortran_order': False, 'shape': (3,), }            \n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [125]:
pos.pos[0] = 11 # Update the array

In [126]:
pos.pos # Updated

array([ 11.,   0.,   0.])

In [127]:
pos._pos # Doesn't get updated!

b"\x93NUMPY\x01\x00F\x00{'descr': '<f8', 'fortran_order': False, 'shape': (3,), }            \n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [113]:
%%timeit
pos.pos

327 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [109]:
class simple_class(object):
    def __init__(self):
        self.pos = np.zeros(3)

In [110]:
obj = simple_class()

In [114]:
%%timeit
obj.pos

47.5 ns ± 0.789 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
